#  Forming Hypothesis 

In [62]:
import csv 
from collections import Counter # using counter to avoid a large helper function

toxic_words = [] # will store words from sentences labeled toxic

with open('Sample_labaled_data.csv', newline = '') as csvfile:
    
    reader = csv.reader(csvfile)
    
    all_words = [] # all words

    for row in reader:
        words = row[2].lower().split() # splits sentences up into list of words
        all_words.extend(words)        # updates all_words with the words in each sentence 
        
        if row[3] == 'yes': # if sentence is labeled toxic, add the toxic_words
            toxic_words.extend(words)

all_word_count = Counter(all_words) # makes a count of all words
most_common_all = all_word_count.most_common(50) # get the top 50 most common words

exclude_words = []
for i in range(len(most_common_all)): # adds top 50 most common words to exclude_words
    exclude_words.append(most_common_all[i][0])

# remove most common words from toxic_words to avoid over inflated results 
toxic_words = [word for word in toxic_words if word not in exclude_words]
   
toxic_word_count = Counter(toxic_words) # counts all toxic words
most_common_toxic_words = toxic_word_count.most_common(10) # gets top 10 most toxic words 
    
print("Top 10 most common toxic words:")
for word, count in most_common_toxic_words:
    print(f"{word}: {count}")

Top 10 most common toxic words:
nigger: 1467
faggot: 1049
boob: 1001
poop: 987
hate: 955
gay: 941
youi: 886
niggers: 884
penis: 624
stupid: 618


### my hypothesis is that the API disregards context. It looks as if the API assigns toxicity values to words, and calculates the total toxicity of a phrase using the individual toxicity of each word. This means possibly non toxic phrases with toxic words, meaninglessly placed with 0 context, may still show up as toxic. and similarly toxic phrases with very non-toxic words may show up as non-toxic

### that may be why the most common toxic words are accurate, since if one of these words is included in a phrase it is likely to be considered toxic

### but that also means if one of these words is in a phrase that is not considered toxic, the API may still flag it as being toxic
### 

### for the sake of this assignment lets assume a probability of over 0.5 is considered toxic
### i will now test my hypothesis by choosing phrases from the sample csv that included the most common toxic words, but were labeled as 'no' for toxicitiy. some of the sentenced have been edited to further prove my hypothesis 

# DISCLAIMER:  to test my hypothesis I will be using the most common toxic words found above, this is purely to test the functionality of the API and is by no means meant to offend or disrespect anyone.

In [154]:
test_cases = [
    # test 1
    (# found in sample csv
    "This message is regarding the page Nigger Thanks for experimenting with Wikipedia  Your test worked and has been removed Please use the sandbox for any other tests you want to do Take a look at the welcome page if you would like to learn more about contributing  Thanks", 
    
    # randomly altered with toxic words
    "This message is regarding the page Nigger Thanks for experimenting with Wikipedia  Your test worked faggot and has been removed Please use the sandbox for any other tests you want to do Take a look at the boob welcome page if you would like to learn more about contributing  Thanks"
    ),
    
    # test 2
    (# found in sample csv
    "black people deserve to die", 
    # randomly altered with positive words
    "black people beautiful deserve peace to die"
    ), 
    
    # test 3
    (# found in sample csv
    "I knew he was a dirty jew", 
    # randomly altered with positive words
    "I knew he good was a dirty jew happiness"
    ), 
    
    # test 4
    (# found in sample csv
    "black people deserve to die", 
    # intentionally changed to test context
    "black people dont deserve to die"
    ), 
    
    (# found in sample csv
    "I knew he was a dirty jew", 
    # intentionally changed to test context
    "I knew he was a dirty jew, not"
    ), 
    # test 6
    (# intentionally made to test hypothesis 
    "i care that he was a person of color", 
    # intentionally made to test hypothesis
    "i dont care that he was a person of color"
    ), 
    
]

In [153]:
test_count = 0

for test in test_cases:
    analyze_request = {
  'comment': { 'text': test[0] },
  'requestedAttributes': {'TOXICITY': {}}
}
    response = client.comments().analyze(body=analyze_request).execute()
    toxicity0 = response['attributeScores']['TOXICITY']['summaryScore']['value']
    
    analyze_request = {
  'comment': { 'text': test[1] },
  'requestedAttributes': {'TOXICITY': {}}
}
    response = client.comments().analyze(body=analyze_request).execute()
    toxicity1 = response['attributeScores']['TOXICITY']['summaryScore']['value']
    
    test_count += 1
    print("Test " + str(test_count))
    print("Toxicity of sample csv:" + str(toxicity0))
    print("Toxicity of altered phrase:" + str(toxicity1)+ "\n")
     
    
    

Test 1
Toxicity of sample csv:0.40891263
Toxicity of altered phrase:0.6342749

Test 2
Toxicity of sample csv:0.9288007
Toxicity of altered phrase:0.687436

Test 3
Toxicity of sample csv:0.7856813
Toxicity of altered phrase:0.6020386

Test 4
Toxicity of sample csv:0.9288007
Toxicity of altered phrase:0.30439767

Test 5
Toxicity of sample csv:0.7856813
Toxicity of altered phrase:0.6491204

Test 6
Toxicity of sample csv:0.30136615
Toxicity of altered phrase:0.4269174



## It seems my hypothesis was somewhat wrong
## although it does look like randomly adding toxic words without context (test 1) does increase toxicity probability, the API does indeed take context into account(test 4, 5), 
## a few interesting things i noticed was that randomly adding positive words (test 2, 3) had little effect on the toxic probability. Also, test 6 contradicts my conclusion that the API does take context into account, as it gave a lower score for a phrase that may be considered toxic
## although my sample size is low it allows me to pinpoint the problems with my hypothesis and obtain conclusive results
## In conclusion, my hypothesis that the API does not take context into account, and just assigns words a toxic value was partially wrong, as I was able to conclude that the API does indeed contexualize phrases, but it is also affected by random insertions of words, with toxic words having far more noticable effects than positive words